In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

## Regresja logistyczna 

### <span style="color:purple"> Model probabilistyczny </span>
Każdy przykład ze zbioru danych $x_i \in X$ ma przypisaną etykietę $y_i \in \{1, \ldots, K \}$.
 <span style="color:purple">Regresja logistyczna </span> jest jednym z klasycznych modeli, który bezpośrednio nadaje się zarówno do klasyfikacji binarnej (dwie klasy: $y_i \in \{0,1\}$), jak i wieloklasowej. 
Ten model jest o tyle ważny, że stanowi podstawę modeli klasyfikacyjnych opartych o sieci neuronowe. 

Regresja logistyczna tworzy model probabilistyczny określający prawdopodobieństwo przynależności punktu do poszczególnych klas. 
W tym modelu chcemy dla każdego punktu wyznaczyć tak zwany rozkład a posteriori $p(1|x_i),\ldots,p(K|x_i)$ określający przynależność punktu $x_i$ do każdej z $K$ klas. Jako finalną decyzję o klasyfikacji przyjmujemy tę najbardziej prawdopodobną, czyli:

$$
c(x) = \arg \max_{k \in \{1, \dots, K\}} p(k | x_i).
$$

Żeby zbudować taki model, musimy sparametryzować prawdopodobieństwa a posteriori, a następnie zbudować funkcję kosztu definiującą kryterium optymalizacyjne. W celu sparametryzowania $p(k|\cdot)$, określmy moc przyporządkowania punktu $x$ do klasy $k$, wykorzystując model liniowy postaci:

$$
f_k(x) = w_k^T+b_k \in \mathbb{R}, \quad \text{dla} \quad k \in \{1, \ldots, K\}.
$$

Mając $K$ takich modeli liniowych, możemy wskazać najbardziej prawdopodobną klasę poprzez wyznaczenie największej wartości $f_k(x)$ dla $k \in \{1, \ldots, K\}$.
W celu transformacji tcyh funkcji do prawdopodobieństw wykorzystamy funkcję <span style="color:purple"> softmax </span> postaci:

$$
p(k|x) = \frac{\exp(z_k)}{\sum_{j=1}^{K}\exp(z_j)} \in (0,1),
$$

gdzie $z_k = f_k(x) = w_k^T + b_k.$

### <span style="color:purple"> Funkcja kosztu </span>

Estymacja modelu regresji logistycznej polega na znalezieniu parametrów $w_k, b_k$ dla wszytskich klas. Standardowa metoda polega na maksymalizacji funkcji wiarygodności $\prod_{i=1}^N p(y_i|x_i)$. Biorąc logarytm (z minusem) otrzymujemy problem minimalizacji:

$$
\text{LR}_{w,b}(X,Y) = -\log(X,Y) = - \frac{1}{N} \sum_{i=1}^N \log p (y_i|x_i),
$$

gdzie $w= (w_1, \ldots, w_K)$, $b = (b_1, \ldots, b_K)$. W terminologii sieci neuronowych powyższą funkcję nazywa się często <span style="color:purple">entropią krzyżową (z ang. cross-entropy) </span>. Entropia krzyżowa w połączeniu z funkcją softmax stanowi podstawową funkcję kosztu stosowaną w klasyfikacyjnych sieciach neuronowych. 

W przypadku klasyfikacji binarnej (dwóch klas) musimy jedynie wyznaczyć jedynie $w_1,b_1$, ponieważ wartości  $w_2,b_2$ wyznaczamy korzystajac z definicji prawdopodobieństwa. W tej sytuacji prawdopodobienstwo a posteriori jest zadane za pomocą <span style="color:purple">funkcji sigmoidalnej: </span>
$$
p(1|x) = \frac{\exp(w^Tx+b)}{1+\exp(w^Tx+b)}.
$$

### <span style="color:purple"> Minimalizacja gradientowa - gradient descent </span>
Minimalizacja gradientowa <span style="color:purple">(ang. Gradient Descent, DG)</span> to jedna z najczęściej używanych technik optymalizacji, która polega na iteracyjnym poszukiwaniu minimum funkcji kosztu (straty). Metoda ta jest fundamentalna w kontekście uczenia maszynowego i głębokiego uczenia, szczególnie w kontekście algorytmów takich jak regresja logistyczna czy sieci neuronowe. 

Celem minimalizacji gradientowej jest znalezienie takich wartości parametrów (wag), które minimalizują funkcję kosztu, czyli błędy predykcji modelu w stosunku do rzeczywistych danych. Działanie tej metody polega na obliczeniu gradientu funkcji kosztu względem wag modelu, a następnie „kroczymy” w kierunku przeciwnym do gradientu, aby zmniejszyć wartość funkcji kosztu. 

<span style="color:purple">Kroki algorytmu minimalizacji gradientowej</span>.

1. **Inicjalizacja wag**:  
   Wagi (parametry modelu) są inicjowane losowo.

2. **Obliczenie gradientu**:  
   Dla bieżących wag, obliczamy gradient funkcji kosztu, czyli zestawienie wszystkich jej pochodnych cząstkowych. Gradient to wektor, który wskazuje kierunek, w którym funkcja kosztu rośnie najszybciej.

3. **Aktualizacja wag**:  
   Wagi są aktualizowane poprzez „krok” w kierunku przeciwnym do gradientu:
   
   $$
   w_{\text{new}} = w_{\text{old}} - \eta \cdot \nabla J(w)
   $$
   
   gdzie:
   - $ w_{\text{new}} $ to nowe wagi,
   - $ w_{\text{old}} $ to obecne wagi,
   - $ \eta $ to współczynnik uczenia (learning rate),
   - $\nabla J(w)$ to gradient funkcji kosztu $ J $ względem wag $ w $.

4. **Powtarzanie**:  
   Proces ten jest powtarzany przez określoną liczbę iteracji lub do momentu, gdy zmiany wag staną się minimalne.


### <span style="color:purple">Uwaga</span>
Wybór punktu startowego oraz współczynnika uczenia (learning rate) ma znaczenie dla działania algorytmu. 


### Zadanie 1.
Zaimplementuj funkcje sigmoidalną oraz softmax. Używając biblioteki matplotlib przedstaw wykresy tych funkcji dla przykładowych danych. 

In [ ]:
# Przykład danych
x = np.linspace(-10, 10, 100)


### Zadanie 2 — Implementacja regresji logistycznej z ręcznym liczeniem gradientów

Twoim zadaniem jest zaimplementowanie klasy `LogisticRegression` od podstaw, **bez używania automatycznego różniczkowania** (`autograd`, `.backward()`, `.grad` itd.). W tym celu trzeba napisać następujące funkcje:
1. Funkcję kosztu modelu regresji logistycznej `loss(X, y)`, według następujących kroków:
    * Policz model liniowy $z = w^Tx + b$
    * Na wektorze $z$ zaimplementuj funkcję $\hat{y} = \sigma(z) = \frac{1}{1 + \exp(-z)}$.
    * Policz entropię krzyżową pomiędzy predykcjami $\hat{y}$ a etykietami $y$ zadaną przez:
    $\frac{1}{N} \sum_i - (1 - y_i) \ln (1 - \hat{y}_i) - y_i \ln \hat{y}_i$
2. Funkcję `predict_proba(X)` zwracającą dla każdego $x_i \in X$ zadane przez nasz model prawdopodobieństwo $\hat{p}(y = 1 \mid x_i)$. 
3. Funkcję `predict(X)` zwracającą dla każdego $x_i \in X$ przewidywaną etykietę (tzn. $0$ albo $1$). Zwracana etykieta powinna być typu `float`.
4. Funkcję `fit`, w której należy:
- policzyć predykcje `y_hat` przy użyciu funkcji sigmoidalnej,
- obliczyć błąd `error = y_hat - y`,
- obliczyć gradienty względem wag i biasu, **Nie używaj** `.backward()` ani `.grad` — wszystkie pochodne mają być wyznaczone **analitycznie**!
- zaktualizować parametry modelu ręcznie przy użyciu algorytmu gradientowego.

**UWAGA** Nie można korzystać z funkcji PyTorcha do liczenia entropii krzyżowej (np. `torch.nn.BCELoss`) ani sigmoidy (np.`torch.nn.functional.Sigmoid`).


In [ ]:
import sys
from collections import namedtuple

if sys.version_info[0] < 3:
    raise Exception("Must be using Python 3")
elif sys.version_info[1] < 7:
    Dataset = namedtuple(
        "Dataset",
        ["data", "target", "target_names", "filename"],
    )
    Dataset.__new__.__defaults__ = (None,) * len(Dataset._fields)
else:
    Dataset = namedtuple(
        "Dataset", ["data", "target", "target_names", "filename"], defaults=(None, None, None, None)
    )

def get_classification_dataset_1d() -> Dataset:
    torch.manual_seed(8)
    X = torch.cat(
        [
            torch.randn(10, 1) * 3 + 10,
            torch.randn(10, 1) * 3 + 1,
        ]
    )

    y = torch.cat([torch.zeros(10), torch.ones(10)])
    return Dataset(X, y)


def get_classification_dataset_2d() -> Dataset:
    torch.manual_seed(4)
    X = torch.cat(
        [
            torch.randn(50, 2) * 2 + torch.tensor([4.0, 2.0]),
            torch.randn(50, 2) * 0.5 + torch.tensor([2.0, -4.0]),
        ]
    )

    y = torch.cat([torch.zeros(50), torch.ones(50)])
    return Dataset(X, y)

In [ ]:
# Przygotujmy datasety i funkcje pomocnicze
dataset_1d = get_classification_dataset_1d()
dataset_2d = get_classification_dataset_2d()


def calculate_accuracy(
    logistic_reg: "LogisticRegression", X: torch.Tensor, y: torch.Tensor
) -> float:
    preds = logistic_reg.predict(X)
    correct_n = (preds == y).float().sum().item()
    return float(correct_n / len(y))


def plot_dataset_1d(logistic_reg: "LogisticRegression", dataset_1d: Dataset) -> None:
    plt.scatter(dataset_1d.data[:10], [0.5] * 10, c="purple", label="0")
    plt.scatter(dataset_1d.data[10:], [0.5] * 10, c="yellow", label="1")
    linspace = torch.linspace(-7.5, 15, steps=100).view(-1, 1)
    plt.plot(
        linspace.numpy().ravel(),
        logistic_reg.predict_proba(linspace).detach().numpy(),
        label="p(y=1 | x)",
    )
    plt.legend()
    plt.show()


def plot_dataset_2d(logistic_reg: "LogisticRegression", dataset_2d: Dataset) -> None:
    plt.scatter(dataset_2d.data[:50, 0], dataset_2d.data[:50, 1], c="purple", label="0")
    plt.scatter(dataset_2d.data[50:, 0], dataset_2d.data[50:, 1], c="yellow", label="1")

    linspace_x = torch.linspace(-4, 7, steps=100)
    linspace_y = (-logistic_reg.bias - logistic_reg.weight[0] * linspace_x) / logistic_reg.weight[1]

    linspace_y = linspace_y.detach().numpy()
    plt.plot(linspace_x.detach().numpy(), linspace_y, label="Granica decyzyjna")
    plt.legend()

In [ ]:
# your code

In [ ]:
logistic_reg = LogisticRegression(1)
logistic_reg.fit(dataset_1d.data, dataset_1d.target, lr=1e-3, num_steps=int(2e4))
acc = calculate_accuracy(logistic_reg, dataset_1d.data, dataset_1d.target)
print("Accuracy", acc)

plot_dataset_1d(logistic_reg, dataset_1d)

In [ ]:
logistic_reg = LogisticRegression(2)
logistic_reg.fit(dataset_2d.data, dataset_2d.target, lr=1e-2, num_steps=int(2e4))
acc = calculate_accuracy(logistic_reg, dataset_2d.data, dataset_2d.target)
print("Accuracy", acc)

plot_dataset_2d(logistic_reg, dataset_2d)

### Zadanie 3.

Wykorzystując zaimplementowany algorytm minimalizacji gradientowej z poprzedniego zadania sprawdź, jak wybór punktu początkowego wpływa na działanie algorytmu.
Twoim celem jest zminimalizowanie funkcji $f(x) = x^4 - 4 \cdot x^3 + 3 \cdot x^2 $ za pomocą algorytmu gradient descent.

W tym celu należy wykonać następujące kroki:

1. Napisać funkcję $ f(x) = x^4 - 4 \cdot x^3 + 3 \cdot x^2 $, która będzie reprezentować nasz obiekt do minimalizacji.
2. Obliczyć pochodną tej funkcji.
3. Zaimplemntować gradient descent.
4. Spróbuj różnych wartości punktu początkowego i sprawdź, jak zmienia się zbieżność algorytmu.

In [ ]:
# Funkcja, którą chcemy minimalizować za pomocą GD
def f(x):
    ...

# Pochodna tej funkcji
def df(x):
    ...

# Implementacja algorytmu minimalizacji gradientowej
def gradient_descent(starting_point, learning_rate, iterations):
    ...
    
x_vals = np.linspace(-1, 3, 400)
y_vals = f(x_vals)

# Parametry minimalizacji gradientowej
learning_rate = 0.01
iterations = 30

# Punkty startowe dla dwóch różnych trajektorii
starting_point_1 = -1  
starting_point_2 = 3  

# Obliczanie trajektorii dla obu punktów startowych
trajectory_1 = gradient_descent(starting_point_1, learning_rate, iterations)
trajectory_2 = gradient_descent(starting_point_2, learning_rate, iterations)

fig, axs = plt.subplots(1, 2, figsize=(14, 6))


axs[0].plot(x_vals, y_vals, label="Funkcja f(x)", color="blue")
axs[0].plot(trajectory_1, f(np.array(trajectory_1)), label=f"Trajektoria (start {starting_point_1})", color="red", marker='o')
axs[0].set_title("Minimum lokalne")
axs[0].set_xlabel("x")
axs[0].set_ylabel("f(x)")
axs[0].grid(True)
axs[0].legend()

axs[1].plot(x_vals, y_vals, label="Funkcja f(x)", color="blue")
axs[1].plot(trajectory_2, f(np.array(trajectory_2)), label=f"Trajektoria (start {starting_point_2})", color="green", marker='o')
axs[1].set_title("Minimum globalne")
axs[1].set_xlabel("x")
axs[1].set_ylabel("f(x)")
axs[1].grid(True)
axs[1].legend()

plt.tight_layout()
plt.show()


### Zadanie 4 

Rozważ funkcję:

$$
f(x_1, x_2) = \log(1 + x_1^2 + x_2^2) + \exp(0.1 x_1 x_2)
$$

Twoim celem jest zminimalizowanie tej funkcji przy użyciu **ręcznie zaimplementowanego gradient descent**, **bez automatycznego różniczkowania**.
Wykonaj następujące kroki:

1. Zaimplementuj funkcję `f(x)`, przyjmującą `x = [x1, x2]`.
2. Ręcznie oblicz gradient tej funkcji
3. Zaimplementuj algorytm gradient descent.
4. Przetestuj działanie algorytmu dla różnych punktów startowych.
5. Przedstaw wizualizację ścieżek zbieżności algorytmu.